In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv(r"C:\Users\User\Downloads\energydata_complete.csv")
df.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,...,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,...,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,...,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,...,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,...,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,...,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


From the dataset, fit a linear model on the relationship between the temperature in the living room in Celsius (x = T2) and the temperature outside the building (y = T6). What is the R^2 value in two d.p.?

In [47]:
#splitting data
msk = np.random.rand(len(df))<0.7 
train = df[msk]
test = df[~msk]

#model data
from sklearn.linear_model import LinearRegression
regr = linear_model.LinearRegression()
train_x = np.asanyarray(train[['T2']])
train_y = np.asanyarray(train[['T6']])
regr.fit(train_x,train_y)

#calculate r2 score
from sklearn.metrics import r2_score
test_x = np.asanyarray(test[['T2']])
test_y = np.asanyarray(test[['T6']])
pred_values = regr.predict(test_x)
r2_score = r2_score(test_y, pred_values)
round(r2_score, 2)

0.64

Normalize the dataset using the MinMaxScaler after removing the following columns: [“date”, “lights”]. The target variable is “Appliances”. Use a 70-30 train-test set split with a random state of 42 (for reproducibility). Run a multiple linear regression using the training set and evaluate your model on the test set. Answer the following questions:

What is the Mean Absolute Error (in two decimal places)?

In [27]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
scaler = MinMaxScaler()
#remove columns
df1= df.drop(['date','lights'],axis=1)

#normalize dataset
normalised_df = pd.DataFrame(scaler.fit_transform(df1),columns=df1.columns)
features_df = normalised_df.drop(columns = ['Appliances'])
target = normalised_df['Appliances']

#split dataset
x_train, x_test, y_train, y_test = train_test_split(features_df, target, test_size = 0.3, random_state = 42)
#model data
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(x_train,y_train)
predicted_values = linear_model.predict(x_test)

#calculate MAE
mae = mean_absolute_error(y_test, predicted_values)
round(mae,3)

0.05

What is the Residual Sum of Squares (in two decimal places)?

In [34]:
rss = np.sum(np.square(predicted_values - y_test))
round(rss, 2)

45.35

What is the Root Mean Squared Error (in three decimal places)?

In [28]:
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, predicted_values))
round(rmse, 3)

0.088

What is the Coefficient of Determination (in two decimal places)?

In [29]:
#the coefficient of determination is also known as the R-squared

from sklearn.metrics import r2_score
r2_score = r2_score(y_test, predicted_values)
round(r2_score, 2)

0.15

Obtain the feature weights from your linear model above. Which features have the lowest and highest weights respectively?

In [36]:
def get_weights(model, feat, col_name):
    weights = pd.Series(model.coef_, feat.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

linear_model_weights = get_weights(linear_model, x_train, 'Linear_Model_Weight')
linear_model_weights

,Features,Linear_Model_Weight
0,RH_2,-0.456698
1,T_out,-0.321860
2,T2,-0.236178
3,T9,-0.189941
4,RH_8,-0.157595
5,RH_out,-0.077671
6,RH_7,-0.044614
7,RH_9,-0.039800
8,T5,-0.015657
9,T1,-0.003281


RH_2 and RH_1 have the lowest and highest weights respectively

Train a ridge regression model with an alpha value of 0.4. Is there any change to the root mean squared error (RMSE) when evaluated on the test set?

In [37]:
from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha = 0.4)
ridge_reg.fit(x_train,y_train)
predicted = ridge_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
round(rmse, 3)

0.088

There is no change to the RMSE .

Train a lasso regression model with an alpha value of 0.001 and obtain the new feature weights with it. How many of the features have non-zero feature weights?

In [38]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha = 0.001)
lasso_reg.fit(x_train,y_train)
lasso_weights = get_weights(lasso_reg, x_train, 'Lasso_Weight')
lasso_weights

,Features,Lasso_Weight
0,RH_out,-0.049557
1,RH_8,-0.000110
2,T1,0.000000
3,Tdewpoint,0.000000
4,Visibility,0.000000
5,Press_mm_hg,-0.000000
6,T_out,0.000000
7,RH_9,-0.000000
8,T9,-0.000000
9,T8,0.000000


From the results above, 4 features have non-zero weights.

What is the new RMSE with the lasso regression? (Answer should be in three (3) decimal places)

In [39]:
predicted = lasso_reg.predict(x_test)
rmse = np.sqrt(mean_squared_error(y_test, predicted))
round(rmse, 3)

0.094